In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import classification_report, roc_auc_score, RocCurveDisplay, confusion_matrix, roc_curve, auc
from sklearn.calibration import calibration_curve
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
from statsmodels.stats.outliers_influence import variance_inflation_factor
from lightgbm import LGBMClassifier
import shap
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import roc_utils as ru
import joblib

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
warnings.filterwarnings(action = 'ignore')
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20) 

In [3]:
df = pd.read_csv("../data/promise_imputado.csv", sep = ";")

FileNotFoundError: [Errno 2] No such file or directory: '../data/promise_imputado.csv'

In [ ]:
df.head()

In [ ]:
df["obito_uti"].value_counts()

In [ ]:
#remove patients without outcomes
#remove nans
#df.dropna(inplace = True)

In [ ]:
categorical_features = ["sexo", "numero_cirurgias", "padronizacao_cirurgias", "fechamento_aae", "has", "dm2", "ic_fer", "irc", "fib_atrial", 
                        "avc", "iam", "rm_previa", "atc_previa", "endocardite", "vasopress", "balao", "hemocomponente", 
                        "eco_disf_vd"]

numerical_features = ["idade", "imc", "tempo_cec_min", "hb_pre_op", "hb_pos_op", "cr_pre_op", "cr_pos_op", "tempo_vm_horas", 
                      "pico_tropo", "pico_lactato_24h", 
                      "eco_feve"]

outcome = ["obito_uti"]

In [ ]:
df[numerical_features].dtypes

In [ ]:
# #dummy variable
# def to_dummy(x):
#     if x == 1:
#         return 1
#     elif x == 2:
#         return 0
# for col in categorical_features:
#     df[col] = df[col].apply(lambda x: to_dummy(x))
    
#scaleing numerical features
scaler = StandardScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

In [ ]:
df["obito_uti"].value_counts()

In [ ]:
#lead wtih imbalanced data
cure = df[df["obito_uti"] == 0]
death = df[df["obito_uti"] == 1]
cure = cure.sample(n = len(death), random_state = 42)
backup_data = df
df = pd.concat([cure, death])

In [ ]:
features_df = df[[col for col in df.columns if col != "obito_uti"]]
target = df["obito_uti"]

In [ ]:
#remove autocorrelated variable
#check for VIF
vif_data = pd.DataFrame()
vif_data["feature"] = features_df.columns

In [ ]:
#perform imputation
cat_imputer = SimpleImputer(strategy = 'most_frequent', missing_values = np.nan)
for col in categorical_features:
    cat_imputer.fit(features_df[[col]])
    features_df[col] = cat_imputer.transform(features_df[[col]])
    
    
#imput numerical col

num_imputer = SimpleImputer(strategy = 'median', missing_values = np.nan)
for col in numerical_features:
    num_imputer.fit(features_df[[col]])
    features_df[col] = num_imputer.transform(features_df[[col]])

In [ ]:
cols = []
value = []
cols_to_remove = []
for col in features_df.columns:
    miss_prop = features_df[col].isna().sum()/features_df.shape[0]
    cols.append(col)
    value.append(miss_prop)
    print("{} missing proportion {}".format(col, miss_prop))
    if miss_prop > 0.35:
        cols_to_remove.append(col) 
    
#missing = pd.DataFrame.from_dict({"colum": cols, "missing": value})
#missing.to_csv("../results/Mising_porp.csv", index = False)
cols_to_remove

In [ ]:
vif_data["VIF"] = [variance_inflation_factor(features_df.values, i)
                          for i in range(len(features_df.columns))]
  
print(vif_data)

In [ ]:
#remove columns with vif greater than 5
cols_to_remove = []
for col in vif_data["feature"]:
    if vif_data[vif_data["feature"] == col]["VIF"].values[0] >= 5.0:
        cols_to_remove.append(col)

In [ ]:
features_df.drop(columns = cols_to_remove, inplace = True)

#remove sts and euro
features_df.drop(columns = ["euro"], inplace = True)

In [ ]:
classifier = RandomForestClassifier(random_state = 42)
features_to_keep = RFE(classifier, n_features_to_select = 7, step = 1)

In [ ]:
features_to_keep.fit(features_df, target.values.ravel())

In [ ]:
features_to_keep = [f for f in features_df.columns[features_to_keep.support_]]
features_to_keep

In [ ]:
param_grid  = {
    "n_estimators": [100,200,500,700,1000],
    "max_depth": [2,4,6,8,10,12,14,16]
}

In [ ]:
repeated_cv = RepeatedKFold(n_splits = 10, n_repeats = 10, random_state = 42)
model_to_fit = GridSearchCV(classifier, param_grid = param_grid, cv = repeated_cv, n_jobs = 30, scoring = "roc_auc")

In [ ]:
model_to_fit.fit(features_df[features_to_keep], target)

In [ ]:
print("The roc_auc in train set was :", model_to_fit.best_score_)
print("Best parameters found was: ", model_to_fit.best_params_)

In [ ]:
#compute metrics
predicted = model_to_fit.predict(features_df[features_to_keep])
predicted_proba = model_to_fit.predict_proba(features_df[features_to_keep])
tn, fp, fn, tp = confusion_matrix(y_true = target, y_pred = predicted).ravel()
print(classification_report(y_true = target, y_pred = predicted))

In [ ]:
#compute specificity
print(tn/(tn + fp))

#compute sensitivity
print(tp/(tp + fn))


#compute ppv
print(tp/(tp+fp))
      
#compute pnv
print(tn/(fn+tn))

In [ ]:
79/97

In [ ]:
print("Original ROC area: {:0.3f}".format(roc_auc_score(target, predicted_proba[:,1])))

In [ ]:
y_true = target.values
y_pred = predicted_proba[:,1]
n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []
fpr_values = []
tpr_values = []
rng = np.random.RandomState(rng_seed)

for i in range(n_bootstraps):
     indices = rng.randint(0, len(y_true), len(y_true))
     score = roc_auc_score(y_true[indices], y_pred[indices])
     fpr, tpr, _ = roc_curve(y_true[indices],  y_pred[indices])
     fpr_values.append(fpr)
     tpr_values.append(tpr)
     bootstrapped_scores.append(score)
  
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

In [ ]:
confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

In [ ]:
n_samples = 1000
fig, ax = plt.subplots()
fig.set_figheight(15)
fig.set_figwidth(20)
ru.plot_roc_bootstrap(X = y_true, y = predicted, pos_label = True,
                      n_bootstrap = n_samples,
                      random_state = 15)
plt.savefig("../results/ROC_plot.pdf")

In [ ]:
#calibration plot
prob_y, prob_x = calibration_curve(y_true = target, y_prob = predicted_proba[:,1], n_bins = 10)
fig, ax = plt.subplots()
fig.set_figheight(15)
fig.set_figwidth(20)
plt.plot(prob_x, prob_y, marker = 'o', linewidth = 2, label = "rf")
lines = mlines.Line2D([0,1],[0,1], color = "black", linewidth = 2)
transform = ax.transAxes
lines.set_transform(transform)
ax.add_line(lines)
ax.set_xlabel("Predicted probability", fontsize = 32)
ax.set_ylabel("True probability in each bin", fontsize = 32)
plt.savefig("../results/Calibration_plot.pdf")

In [ ]:
joblib.dump(model_to_fit.best_estimator_, "../results/random_forest_model.joblib")

In [ ]:
loaded_rf = joblib.load("../results/random_forest_model.joblib")

In [ ]:
#include a column in data
for col in categorical_features:
    cat_imputer.fit(backup_data[[col]])
    backup_data[col] = cat_imputer.transform(backup_data[[col]])
    
    
#imput numerical col

num_imputer = SimpleImputer(strategy = 'median', missing_values = np.nan)
for col in numerical_features:
    num_imputer.fit(backup_data[[col]])
    backup_data[col] = num_imputer.transform(backup_data[[col]])
backup_data["predicted_rf"] = model_to_fit.predict_proba(backup_data[[
 'idade',
 'imc',
 'cr_pre_op',
 'tempo_vm_horas',
 'pico_tropo',
 'pico_lactato_24h',
 'eco_feve']])[:,1]
backup_data.to_csv("../results/2023_11_13_data.csv", index = False)

In [ ]:
print("Done")